![Imagen](itd.png)

# Analisis De Datos Masivos
## Práctica de laboratorio Detección de anomalías en medidores de Internet
    
## Nombre Del Docente
### Jose Gabriel Rodriguez Rivas
    
## Francisco javier Favela Najera 

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('rpi_data_processed.csv')

# Initialize dataframe df_rates
df_rates = df.drop(['Ping (ms)', 'Date', 'Time'], axis=1)

# Rename the download and
# upload columns of df_rates
lookup = {'Download (Mbit/s)': 'download_rate', 
          'Upload (Mbit/s)': 'upload_rate'}
df_rates = df_rates.rename(columns = lookup)

# Calculate ping_rate
ping_rate = 1. / df['Ping (ms)']

# Convert ping_rate to 1/seconds
ping_rate = 1000. * ping_rate

# Add a column to complete the task
df_rates['ping_rate'] = ping_rate

In [4]:
df_rates.describe()

,download_rate,upload_rate,ping_rate
count,672.000000,672.000000,672.000000
mean,90.456295,13.696815,48.662648
std,3.463351,1.081080,6.033436
min,58.480000,6.210000,13.067113
25%,90.220000,13.640000,45.792264
50%,91.565000,13.980000,49.212599
75%,92.070000,14.180000,52.319042
max,92.870000,14.550000,61.854395
